In [1]:
%pwd

'c:\\Users\\seoju\\OneDrive\\문서\\workspace\\ESG 2024 prac'

### Yolo

In [1]:
import cv2
from ultralytics import YOLO

def process_video(input_path, output_path, duration=60):  # 60초만 처리
    model = YOLO('yolov8n.pt')
    
    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    frame_count = 0
    max_frames = duration * fps
    
    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        
        results = model(frame)
        annotated_frame = results[0].plot()
        
        out.write(annotated_frame)
        frame_count += 1
        
        if frame_count % fps == 0:  # 매 초마다 진행 상황 출력
            print(f"Processed {frame_count // fps} seconds")
    
    cap.release()
    out.release()
    print("Video processing completed")

# 사용 예
input_video = "output/ccm_youtube.mp4"
output_video = "output/output_video.mp4"
process_video(input_video, output_video, duration=60)  # 60초만 처리

100%|██████████| 6.25M/6.25M [00:00<00:00, 37.7MB/s]



0: 640x384 7 persons, 1 tie, 1 frisbee, 1 potted plant, 1 clock, 189.9ms
Speed: 9.2ms preprocess, 189.9ms inference, 12.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 7 persons, 1 tie, 1 frisbee, 1 potted plant, 1 clock, 109.5ms
Speed: 3.0ms preprocess, 109.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 7 persons, 1 tie, 1 potted plant, 1 clock, 113.5ms
Speed: 1.0ms preprocess, 113.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 7 persons, 1 tie, 1 potted plant, 1 clock, 100.1ms
Speed: 2.0ms preprocess, 100.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 7 persons, 1 tie, 1 potted plant, 1 clock, 106.5ms
Speed: 1.0ms preprocess, 106.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 7 persons, 1 tie, 1 potted plant, 1 clock, 117.5ms
Speed: 2.0ms preprocess, 117.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384

### Face Recognition

In [4]:
import face_recognition
import cv2
import numpy as np

def process_video(known_face_path, input_video_path, output_video_path):
    # 알려진 얼굴 이미지 로드 및 인코딩
    known_image = face_recognition.load_image_file(known_face_path)
    known_face_encoding = face_recognition.face_encodings(known_image)[0]

    # 비디오 파일 열기
    video_capture = cv2.VideoCapture(input_video_path)
    
    # 비디오 정보 가져오기
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # 출력 비디오 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    frame_count = 0

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        # 프레임 크기를 줄여 처리 속도 향상 (선택사항)
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # BGR을 RGB로 변환 (face_recognition은 RGB를 사용)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        # 얼굴 위치 찾기
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations, num_jitters=1)

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # 알려진 얼굴과 비교
            matches = face_recognition.compare_faces([known_face_encoding], face_encoding)
            
            name = "Unknown"
            
            # 일치하는 얼굴이 있으면 표시
            if True in matches:
                name = "침착맨"

            # 좌표를 원래 프레임 크기로 조정
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # 얼굴 주위에 박스 그리기
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # 이름 표시
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

        # 프레임 저장
        out.write(frame)

        frame_count += 1
        if frame_count % fps == 0:  # 매 초마다 진행 상황 출력
            print(f"Processed {frame_count // fps} seconds")

    # 정리
    video_capture.release()
    out.release()
    print("Video processing completed")

known_face_path = "ccm.jpg"  # 인식하고자 하는 사람의 얼굴 이미지
input_video_path = "output/ccm_youtube.mp4"  # 처리할 비디오 파일
output_video_path = "output/face_output.mp4"  # 결과 비디오 파일

process_video(known_face_path, input_video_path, output_video_path)

Processed 1 seconds
Processed 2 seconds
Processed 3 seconds
Processed 4 seconds
Processed 5 seconds
Processed 6 seconds
Processed 7 seconds
Processed 8 seconds
Processed 9 seconds
Processed 10 seconds
Processed 11 seconds
Processed 12 seconds
Processed 13 seconds
Processed 14 seconds
Processed 15 seconds
Processed 16 seconds
Processed 17 seconds
Processed 18 seconds
Processed 19 seconds
Processed 20 seconds
Processed 21 seconds
Processed 22 seconds
Processed 23 seconds
Processed 24 seconds
Processed 25 seconds
Processed 26 seconds
Processed 27 seconds
Processed 28 seconds
Processed 29 seconds
Processed 30 seconds
Processed 31 seconds
Processed 32 seconds
Processed 33 seconds
Processed 34 seconds
Processed 35 seconds
Processed 36 seconds
Processed 37 seconds
Processed 38 seconds
Processed 39 seconds
Processed 40 seconds
Processed 41 seconds
Processed 42 seconds
Processed 43 seconds
Processed 44 seconds
Processed 45 seconds
Processed 46 seconds
Processed 47 seconds
Processed 48 seconds
P